In [ ]:
import pandas as pd
train=pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
train.head()

In [ ]:
train_labels=pd.read_csv('../input/tabular-playground-series-apr-2022/train_labels.csv')
train_labels.head()

In [ ]:
train =train.merge(train_labels,how='left', on=["sequence"])
train.head()

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')
test.head()

In [ ]:
def aggregated_features(df, aggregation_cols = ['sequence'], prefix = ''):
    agg_strategy = {'sensor_00': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_01': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_02': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_03': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_04': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_05': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_06': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_07': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_08': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_09': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_10': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_11': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_12': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                   }
    group = df.groupby(aggregation_cols).aggregate(agg_strategy)
    group.columns = ['_'.join(col).strip() for col in group.columns]
    group.columns = [str(prefix) + str(col) for col in group.columns]
    group.reset_index(inplace = True)
    
    temp = (df.groupby(aggregation_cols).size().reset_index(name = str(prefix) + 'size'))
    group = pd.merge(temp, group, how = 'left', on = aggregation_cols,)
    return group


In [ ]:
train_merge_data= aggregated_features(train, aggregation_cols = ['sequence', 'subject'])
train_merge_data.head()

In [ ]:
test_merge_data= aggregated_features(test, aggregation_cols = ['sequence', 'subject'])
test_merge_data.head()

In [ ]:
train_subjects_merge_data = aggregated_features(train, aggregation_cols = ['subject'], prefix = 'subject_')
train_subjects_merge_data.head()

In [ ]:
test_subjects_merge_data = aggregated_features(test, aggregation_cols = ['subject'], prefix = 'subject_')
test_subjects_merge_data.head()

In [ ]:
train_merge_data = train_merge_data.merge(train_labels, how = 'left', on = 'sequence')
train_merge_data.head()

In [ ]:
train = train_merge_data.merge(train_subjects_merge_data, how = 'left', on = 'subject')
train.head()

In [ ]:
test = test_merge_data.merge(test_subjects_merge_data, how = 'left', on = 'subject')
test.head()

In [ ]:
train=train.drop(['sequence','subject','size'],axis=1)
train.head()

In [ ]:
test=test.drop(['sequence','subject','size'],axis=1)
test.head()

In [ ]:
train[train.isna().any(axis=1)]

In [ ]:
!pip install flaml
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
from flaml import AutoML
automl = AutoML()

In [ ]:
y = train.pop('state')
X = train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42,shuffle=True, stratify=y)

In [ ]:
automl.fit(X_train, y_train, task="classification",metric='log_loss',time_budget=(3*3600))

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best log_loss on validation data: {0:.4g}'.format(automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, automl.predict(X_train)))

In [ ]:
print(classification_report(y_test, automl.predict(X_test)))

In [ ]:
y_pred = automl.predict_proba(test)[:,1]
y_pred[:5]

In [ ]:
df = pd.DataFrame(y_pred,columns=['state'])
df.head()

In [ ]:
sol=pd.read_csv('../input/tabular-playground-series-apr-2022/sample_submission.csv')
sol.head()

In [ ]:
sol['state']=df['state']
sol.head()

In [ ]:
sol.to_csv('./log_loss.csv',index=False)